In [ ]:
import base64
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, AIMessage
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
chain = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1024)


In [ ]:
image = encode_image("./food.jpeg")


msg = chain.invoke(
    [   AIMessage(
        content="You are a useful bot that is especially good at OCR from images"
    ),
        HumanMessage(
            content=[
                {"type": "text", "text": "Identify all items on the this image which are food related and provide a list of what you see"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image}"
                    },
                },
            ]
        )
    ]
)
print(msg.content)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser


chain = ChatOpenAI(model="gpt-3.5-turbo", max_tokens=1024)
prompt = PromptTemplate.from_template(
"""
You see the following foods in a product list:
{food}
Create a dish using only these ingredients and say what it is called. Only return the name of the dish.
No additional explanation or anything related. Just the name

Example:
'Pizza'
'Sushi'

Output:
"""
)
runnable = prompt | chain | StrOutputParser()

In [ ]:
dish = runnable.invoke({"food": msg.content})
print(dish)

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.images.generate(
  model="dall-e-3",
  prompt=f"A nice candlelight dinner with {dish} for two persons",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url


In [ ]:
print(image_url)